# Project: Semantic similarity extraction using word vectors in Harry Potter dataset

## Exploring the Data
Run the code cell below to load necessary Python libraries.

In [1]:
#future is the missing compatibility layer between Python 2 and Python 3. 
#It allows you to use a single, clean Python 3.x-compatible codebase to 
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

In [2]:
#word encoding
import codecs
#finds all pathnames matching a pattern, like regex
import glob
#log events for libraries
import logging
#concurrency
import multiprocessing
#dealing with operating system
import os
#pretty print, human readable
import pprint
#regular expressions
import re
#natural language toolkit
import nltk
#word 2 vec   (conda install -c anaconda gensim=1.0.1)
import gensim.models.word2vec as w2v
#dimensionality reduction
import sklearn.manifold
#math
import numpy as np
#plotting
import matplotlib.pyplot as plt
#parse dataset
import pandas as pd
#visualization
import seaborn as sns

I:\Anaconda\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Set up logging

In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Download NLTK tokenizer models (only the first time)

In [5]:
##stopwords like the at a an, unnecesasry
##tokenization into sentences, punkt 
##http://www.nltk.org/

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DTILAK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DTILAK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Preparing Corpus
### Load book

In [6]:
#get the book names, matching txt file
book_filenames = sorted(glob.glob("..\Mahabharata_extract-semantic-similarities_Natural-languageprocessing\input\*.txt"))
print("Found book:")
book_filenames

Found book:


['..\\Mahabharata_extract-semantic-similarities_Natural-languageprocessing\\input\\1-18 books combined.txt']

### Combine the book into one string

In [7]:
#initialize raw unicode , we'll add all text to this file in memory
corpus_raw = u""

#for each book, read it, open it in utf 8 format, 
#add it to the raw corpus
for book_filename in book_filenames:
    print("Reading '{0}'...".format(book_filename))
    with codecs.open(book_filename, "r", "utf-8") as book_file:
        corpus_raw += book_file.read()
    print ("Corpus is now {0} characters long".format(len(corpus_raw)))
    print ()

Reading '..\Mahabharata_extract-semantic-similarities_Natural-languageprocessing\input\1-18 books combined.txt'...
Corpus is now 1704913 characters long



### Split the corpus into sentences

In [8]:
#tokenizastion
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

#tokenize into sentences
raw_sentences = tokenizer.tokenize(corpus_raw)

In [9]:
#convert into a list of words
#remove unnnecessary, split into words, no hyphens
#list of words
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words

In [10]:
#sentence where each word is tokenized
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [11]:
# Example
print(raw_sentences[5])
print(sentence_to_wordlist(raw_sentences[5]))

Above all these qualities, he was a devoted servant of Lord Vishnu, and therefore he was given the title, "King of kings".
[u'Above', u'all', u'these', u'qualities', u'he', u'was', u'a', u'devoted', u'servant', u'of', u'Lord', u'Vishnu', u'and', u'therefore', u'he', u'was', u'given', u'the', u'title', u'King', u'of', u'kings']


In [12]:
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens.".format(token_count))

The book corpus contains 293,443 tokens.


### Build word2vec model

In [13]:
# Dimensionality of the resulting word vectors.
# more dimensions, more computationally expensive to train
# but also more accurate
# more dimensions = more generalized
num_features = 300

# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
# more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
# 0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the random number generator, to make the results reproducible.
seed = 1

In [14]:
HP2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [15]:
HP2vec.build_vocab(sentences)

2017-04-08 13:38:40,088 : INFO : collecting all words and their counts
2017-04-08 13:38:40,088 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-04-08 13:38:40,137 : INFO : PROGRESS: at sentence #10000, processed 170274 words, keeping 9385 word types
2017-04-08 13:38:40,174 : INFO : collected 11426 word types from a corpus of 293443 raw words and 17714 sentences
2017-04-08 13:38:40,177 : INFO : Loading a fresh vocabulary
2017-04-08 13:38:40,194 : INFO : min_count=3 retains 5695 unique words (49% of original 11426, drops 5731)
2017-04-08 13:38:40,197 : INFO : min_count=3 leaves 286106 word corpus (97% of original 293443, drops 7337)
2017-04-08 13:38:40,217 : INFO : deleting the raw counts dictionary of 11426 items
2017-04-08 13:38:40,220 : INFO : sample=0.001 downsamples 52 most-common words
2017-04-08 13:38:40,220 : INFO : downsampling leaves estimated 213862 word corpus (74.7% of prior 286106)
2017-04-08 13:38:40,221 : INFO : estimated required memory for

In [16]:
print("Word2Vec vocabulary length:", len(HP2vec.wv.vocab))

Word2Vec vocabulary length: 5695


### word2vec training, this might take a minute

In [17]:
HP2vec.train(sentences)

2017-04-08 13:39:17,424 : INFO : training model with 4 workers on 5695 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=7
2017-04-08 13:39:17,424 : INFO : expecting 17714 sentences, matching count from corpus used for vocabulary survey
2017-04-08 13:39:18,575 : INFO : PROGRESS: at 15.51% examples, 148795 words/s, in_qsize 8, out_qsize 1
2017-04-08 13:39:19,612 : INFO : PROGRESS: at 33.83% examples, 167295 words/s, in_qsize 8, out_qsize 0
2017-04-08 13:39:20,609 : INFO : PROGRESS: at 50.85% examples, 171667 words/s, in_qsize 8, out_qsize 0
2017-04-08 13:39:21,627 : INFO : PROGRESS: at 68.02% examples, 173313 words/s, in_qsize 7, out_qsize 0
2017-04-08 13:39:22,617 : INFO : PROGRESS: at 84.42% examples, 173654 words/s, in_qsize 8, out_qsize 0
2017-04-08 13:39:23,421 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-04-08 13:39:23,461 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-04-08 13:39:23,492 : INFO : worker 

1069556

### Save to file, can be useful later

In [18]:
if not os.path.exists("trained"):
    os.makedirs("trained")

In [19]:
HP2vec.save(os.path.join("trained", "HP2vec.w2v"))

2017-04-08 13:40:53,283 : INFO : saving Word2Vec object under trained\HP2vec.w2v, separately None
2017-04-08 13:40:53,283 : INFO : not storing attribute syn0norm
2017-04-08 13:40:53,283 : INFO : not storing attribute cum_table
2017-04-08 13:40:53,390 : INFO : saved trained\HP2vec.w2v


In [20]:
HP2vec = w2v.Word2Vec.load(os.path.join("trained", "HP2vec.w2v"))

2017-04-08 13:41:10,157 : INFO : loading Word2Vec object from trained\HP2vec.w2v
2017-04-08 13:41:10,263 : INFO : loading wv recursively from trained\HP2vec.w2v.wv.* with mmap=None
2017-04-08 13:41:10,263 : INFO : setting ignored attribute syn0norm to None
2017-04-08 13:41:10,265 : INFO : setting ignored attribute cum_table to None
2017-04-08 13:41:10,265 : INFO : loaded trained\HP2vec.w2v


### Compress the word vectors into 3D space using t-SNE and plot them for further analysis

In [21]:
tsne = sklearn.manifold.TSNE(n_components=3,perplexity=15.0, n_iter=20000,random_state=0)

In [22]:
all_word_vectors_matrix = HP2vec.wv.syn0

In [23]:
import gc
gc.collect()

0

### Train t-SNE, this could take few minute...

In [24]:
all_word_vectors_matrix_3d = tsne.fit_transform(all_word_vectors_matrix)